TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 19
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 29 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.959164,2.319721,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.089483,NaN,NaN,NaN,NaN,NaN,NaN,29.329505,0,3.927468
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.773525,0,3.121150
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.762346,0,2.456387
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.5,3.6,3.8,2.4,6.8,4.071,26.483403,3,3.843080
5,6.245551,1.561388,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.910517,1.0,NaN,NaN,NaN,NaN,NaN,14.697860,1,1.915537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,42.800000,3.057143,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,1.4,2.4,7.8,2.0,6.400,27.831019,1,3.237109
611,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.888838,0,3.401949
621,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.888838,0,3.401949


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=5]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 29 prediction
id,,,,,,,,,,,,,,,,,,,,,
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,26.483403,3,3.843080
6,43.453000,3.103786,3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,25.681306,0,3.495466
7,18.590373,2.065597,1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,-2.000000,NaN,NaN,2.332871,NaN,NaN,NaN,14.503278,3,1.911940
12,9.850582,1.641764,1,NaN,NaN,0.0,0.0,0.0,1.0,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,13.338700,0,1.844786
13,12.872776,1.609097,0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,1.000000,3.096585,1.000000,1.000000,1.000000,NaN,1.000000,14.536428,0,2.064436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,8.300000,1.037500,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.000000,NaN,1.000000,NaN,NaN,1.300000,NaN,15.345105,0,1.906453
461,42.490360,3.035026,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,5.548812,4.418731,2.382085,NaN,5.048812,NaN,NaN,24.137432,0,2.900671
470,31.860497,3.540055,1,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.861196,25.926468,1,3.061764


In [5]:
optimize(df, 85.2, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 84.10000000000001
Expected points: 59.669414043426514


,element_type,web_name,now_cost,games played,gameweek 20 prediction
id,,,,,
146,1,Guaita,46,16.0,4.676792
127,2,Rüdiger,62,18.0,4.558856
135,2,Chilwell,56,6.0,5.382999
237,2,Alexander-Arnold,83,16.0,5.108371
138,3,Mount,77,16.0,5.107860
144,3,Gallagher,61,17.0,4.761030
233,3,Salah,130,18.0,5.319105
419,3,Benrahma,59,18.0,4.709595
420,3,Bowen,65,18.0,4.785216


In [6]:
optimize(df, 85.2, 'adjusted points per game')

Status: Optimal
Cost: 84.2
Expected points per week: 71.15617970404784


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
475,1,Sá,51,18.0,4.136111
135,2,Chilwell,56,6.0,6.154732
142,2,James,64,15.0,5.206565
237,2,Alexander-Arnold,83,16.0,7.149678
256,2,Cancelo,69,18.0,6.002212
138,3,Mount,77,16.0,5.134162
233,3,Salah,130,18.0,7.901461
240,3,Jota,81,17.0,5.852206
250,3,Gündogan,73,13.0,5.521052


In [7]:
#df = df.drop([119])
optimize(df, 85.2, 'xPoints_next10')

Status: Optimal
Cost: 84.7
Expected points per week: 53.284605383872986


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
353,1,Lloris,54,16.0,42.483965
135,2,Chilwell,56,6.0,42.364694
237,2,Alexander-Arnold,83,16.0,42.765125
370,2,Reguilón,53,15.0,41.786613
210,3,Tielemans,64,13.0,41.960623
233,3,Salah,130,18.0,46.362981
265,3,Foden,81,12.0,46.687448
419,3,Benrahma,59,18.0,43.017663
420,3,Bowen,65,18.0,44.036174
